In [1]:
from deeppavlov import build_model
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    
    Doc
)
from natasha import NewsNERTagger

In [2]:
model = build_model('ner_ontonotes_bert_mult', install=True, download=True)

2023-10-18 01:39:12.478 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_mult_torch_crf.tar.gz download because of matching hashes
C:\Users\yarik\anaconda3\envs\ForDeepPavlov\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on ano

In [3]:
text='Александр Сергеевич Пушкин родился 6 июня 1799 года в Москве и происходил из старинного нетитулованного дворянского рода.'

In [4]:
model([text, 'В 1831 г. Пушкин обвенчался с Натальей Гончаровой, с которой впоследствии у них было четверо детей.'])

C:\Users\yarik\anaconda3\envs\ForDeepPavlov\lib\site-packages\torchcrf\__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorCompare.cpp:413.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


[[['Александр',
   'Сергеевич',
   'Пушкин',
   'родился',
   '6',
   'июня',
   '1799',
   'года',
   'в',
   'Москве',
   'и',
   'происходил',
   'из',
   'старинного',
   'нетитулованного',
   'дворянского',
   'рода',
   '.'],
  ['В',
   '1831',
   'г',
   '.',
   'Пушкин',
   'обвенчался',
   'с',
   'Натальей',
   'Гончаровой',
   ',',
   'с',
   'которой',
   'впоследствии',
   'у',
   'них',
   'было',
   'четверо',
   'детей',
   '.']],
 [['B-PERSON',
   'I-PERSON',
   'I-PERSON',
   'O',
   'B-DATE',
   'I-DATE',
   'I-DATE',
   'I-DATE',
   'O',
   'B-GPE',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O'],
  ['O',
   'B-DATE',
   'I-DATE',
   'I-DATE',
   'B-PERSON',
   'O',
   'O',
   'B-PERSON',
   'I-PERSON',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-CARDINAL',
   'O',
   'O']]]

In [5]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

In [6]:
doc = Doc(text)

In [7]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)

In [8]:
sent = doc.sents[0]

In [9]:
sent.morph.print()

           Александр PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
           Сергеевич PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
              Пушкин PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
             родился VERB|Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Mid
                   6 ADJ
                июня NOUN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
                1799 ADJ
                года NOUN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
                   в ADP
              Москве PROPN|Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing
                   и CCONJ
          происходил VERB|Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
                  из ADP
          старинного ADJ|Case=Gen|Degree=Pos|Gender=Masc|Number=Sing
     нетитулованного ADJ|Case=Gen|Degree=Pos|Gender=Masc|Number=Sing
         дворянского ADJ|Case=Gen|Degree=Pos|Gender=Masc|Number=Sing
                рода N

In [10]:
sent.syntax.print()

        ┌──► Александр       nsubj
        │ ┌─ Сергеевич       
        │ └► Пушкин          flat:name
┌─┌─┌───└─┌─ родился         
│ │ │ ┌─┌─└► 6               obl
│ │ │ │ └──► июня            flat
│ │ │ │   ┌► 1799            amod
│ │ │ └──►└─ года            nmod
│ │ │     ┌► в               case
│ │ └────►└─ Москве          obl
│ │       ┌► и               cc
│ └──►┌───└─ происходил      conj
│     │ ┌──► из              case
│     │ │ ┌► старинного      amod
│     └►└─└─ нетитулованного obl
│     │   ┌► дворянского     amod
│     └──►└─ рода            nmod
└──────────► .               punct


In [11]:
ner_tagger = NewsNERTagger(emb)
doc.tag_ner(ner_tagger)
doc.ner.print()

Александр Сергеевич Пушкин родился 6 июня 1799 года в Москве и 
PER───────────────────────                            LOC───   
происходил из старинного нетитулованного дворянского рода.
